In [1]:
import pystac
from pystac import Link, Asset
from datetime import datetime
import os
from osgeo import gdal, ogr, osr
from sklearn.model_selection import train_test_split
import numpy as np
import sys
from pystac.extensions.label import LabelExtension
from pystac.extensions.label import LabelType
from pystac.extensions.label import LabelClasses
from pystac.extensions.label import LabelStatistics
from pystac import read_file
from pystac.extensions.version import ItemVersionExtension
from pystac_client import Client

# sys.path.append(".")
from utils import pixel_to_coords, to_geojson, StarsCopyWrapper, ingest_collection, read_url, ingest_items

os.environ["GDAL_DATA"] = "/opt/conda/envs/env_label/share/gdal"
os.environ["PROJ_LIB"] = "/opt/conda/envs/env_label/share/proj"

import shapely.wkt

In [2]:
collection = read_file("collection/collection.json")

stac_endpoint = "https://stac-api-dev.terradue.com/"

ingest_collection(app_host=stac_endpoint, collection=collection)

2023-08-02 09:19:40.062 | INFO     | utils:post_or_put:197 - https://stac-api-dev.terradue.com/collections 200


In [3]:
# Access to Catalog
URL = "https://earth-search.aws.element84.com/v1/"

headers = []

cat = Client.open(URL, headers=headers, ignore_conformance=True)
cat

/opt/conda/envs/env_label/lib/python3.10/site-packages/pystac_client/client.py:177: FutureWarning: The `ignore_conformance` option is deprecated and will be removed in the next major release. Instead use `set_conforms_to` or `add_conforms_to` to control behavior.
  warnings.warn(


<Client id=earth-search-aws>

In [4]:
# Collection
collections = (["sentinel-2-l2a"],)

# Start and end dates
start_date = datetime.fromisoformat("2023-06-09T00:00:00")
stop_date = datetime.fromisoformat("2023-06-20T00:00:00")

bbox = [-121.698, 37.457, -119.938, 38.295]

# Other metadata
cloud_cover = 30

# Define EPSG code
epsg = "EPSG:4326"

In [5]:
# Query by AOI, start and end date and other params
query = cat.search(
    collections=collections,
    datetime=(start_date, stop_date),
    bbox=bbox,
    query={"eo:cloud_cover": {"lt": 5}},
)

In [6]:
tiles = ["10SFH", "10SFG", "10SGH", "11SKC", "10SGG", "11SKB"]


selected_items = [
    item for item in query.get_all_items() if any(tile in item.id for tile in tiles)
]
selected_items

/opt/conda/envs/env_label/lib/python3.10/site-packages/pystac_client/item_search.py:841: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


[<Item id=S2A_10SFG_20230618_0_L2A>,
 <Item id=S2A_10SGG_20230618_0_L2A>,
 <Item id=S2A_11SKB_20230618_0_L2A>,
 <Item id=S2A_10SFH_20230618_0_L2A>,
 <Item id=S2B_10SFG_20230613_0_L2A>,
 <Item id=S2B_10SFH_20230613_0_L2A>]

In [7]:
label_classes = {
    "name": "class",
    "classes": [
        "NO_DATA",
        "SATURATED_OR_DEFECTIVE",
        "CAST_SHADOWS",
        "CLOUD_SHADOWS",
        "VEGETATION",
        "NOT_VEGETATED",
        "WATER",
        "UNCLASSIFIED",
        "CLOUD_MEDIUM_PROBABILITY",
        "CLOUD_HIGH_PROBABILITY",
        "THIN_CIRRUS",
        "SNOW or ICE",
    ],
}

In [8]:
look_up = {
    0: "NO_DATA",
    1: "SATURATED_OR_DEFECTIVE",
    2: "CAST_SHADOWS",
    3: "CLOUD_SHADOWS",
    4: "VEGETATION",
    5: "NOT_VEGETATED",
    6: "WATER",
    7: "UNCLASSIFIED",
    8: "CLOUD_MEDIUM_PROBABILITY",
    9: "CLOUD_HIGH_PROBABILITY",
    10: "THIN_CIRRUS",
    11: "SNOW or ICE",
}

In [9]:
np.random.seed(42)

for index, item in enumerate(selected_items):
    water_count = 0
    non_water_count = 0

    source = source = gdal.Open(
        f"/vsicurl/{item.get_assets()['scl'].get_absolute_href()}"
    )
    scl = source.GetRasterBand(1)

    
    xy = np.random.randint(1, 5490, size=(10000, 2))

    x_values = []
    y_values = []

    for pos in xy:
        observed = look_up[
            int(
                scl.ReadAsArray(
                    xoff=int(pos[0]), yoff=int(pos[1]), win_xsize=1, win_ysize=1
                )[0][0]
            )
        ]

        if observed in ["NO_DATA", "UNCLASSIFIED", "SATURATED_OR_DEFECTIVE"]:
            continue

        if observed in ["WATER"]:
            water_count = water_count + 1
            if water_count <= 150:
                x_values.append([*pixel_to_coords(source, pos[0], pos[1])])
                y_values.append(observed)
        else:
            non_water_count = non_water_count + 1
            if non_water_count <= 150:
                x_values.append([*pixel_to_coords(source, pos[0], pos[1])])
                y_values.append(observed)

        if water_count > 150:
            break

    to_geojson(f"{item.id}", x_values, y_values)

    label_item = pystac.Item(
        id=f"label_{item.id}",
        geometry=item.geometry,
        bbox=item.bbox,
        datetime=datetime.utcnow(),
        properties={},
    )

    label = LabelExtension.ext(label_item, add_if_missing=True)
    label.apply(
        label_description="Land cover labels",
        label_type=LabelType.VECTOR,
        label_tasks=["segmentation", "regression"],
        label_classes=[LabelClasses(label_classes)],
        label_methods=["manual"],
        label_properties=["class"],
    )
    label.add_source(source_item=item)
    label.add_geojson_labels(f"label-{item.id}.geojson")

    version = ItemVersionExtension(label_item)

    version.apply(version="0.1", deprecated=False)

    label_item.stac_extensions.extend(
        ["https://stac-extensions.github.io/version/v1.2.0/schema.json"]
    )

    #label_item.validate()
    pystac.write_file(label_item, dest_href=f"item-label-{item.id}.json")

    wrapper = StarsCopyWrapper()

    exit_code, stdout, stderr = (
        wrapper.recursivity()
        .output(f"s3://argo-wfs/svv-dataset/{item.id}")
        .config_file("usersettings.json")
        .extract_archive(extract=False)
        .absolute_assets()
        .run(f"file://{os.getcwd()}/item-label-{item.id}.json")
    )

    print(stdout)
    print(f"s3://argo-wfs/svv-dataset/{item.id}/catalog.json")
    catalog = read_url(f"s3://argo-wfs/svv-dataset/{item.id}/catalog.json")
        
    ingest_items(
            app_host=stac_endpoint,
            items=list(catalog.get_all_items()),
            collection=collection,
        )

Stars/2.10.17
Trying to init a root catalog catalog on s3://argo-wfs/svv-dataset/S2A_10SFG_20230618_0_L2A/
Root catalog catalog created at s3://argo-wfs/svv-dataset/S2A_10SFG_20230618_0_L2A/catalog.json
Copy node label_S2A_10SFG_20230618_0_L2A from file:///workspaces/s2-labeled/item-label-S2A_10SFG_20230618_0_L2A.json
[Native] Searching for file:///workspaces/s2-labeled/item-label-S2A_10SFG_20230618_0_L2A.json
[Native] resource found at file:///workspaces/s2-labeled/item-label-S2A_10SFG_20230618_0_L2A.json [application/geo+json]
Starting delivery of assets for labels
Delivering asset labels Unknown file:///workspaces/s2-labeled/label-S2A_10SFG_20230618_0_L2A.geojson (S3Streaming) to s3://argo-wfs/svv-dataset/S2A_10SFG_20230618_0_L2A/label-S2A_10SFG_20230618_0_L2A.geojson...
Delivery asset complete to s3://argo-wfs/svv-dataset/S2A_10SFG_20230618_0_L2A/label-S2A_10SFG_20230618_0_L2A.geojson (40.96 KB)

s3://argo-wfs/svv-dataset/S2A_10SFG_20230618_0_L2A/catalog.json


2023-08-02 09:20:15.985 | INFO     | utils:ingest_items:236 - Post item label_S2A_10SFG_20230618_0_L2A to https://stac-api-dev.terradue.com//collections/ai-extensions-svv-dataset-labels/items
2023-08-02 09:20:17.119 | INFO     | utils:post_or_put:197 - https://stac-api-dev.terradue.com/collections/ai-extensions-svv-dataset-labels/items 200


https://stac-api-dev.terradue.com/collections/ai-extensions-svv-dataset-labels/items/label_S2A_10SFG_20230618_0_L2A
Stars/2.10.17
Trying to init a root catalog catalog on s3://argo-wfs/svv-dataset/S2A_10SGG_20230618_0_L2A/
Root catalog catalog created at s3://argo-wfs/svv-dataset/S2A_10SGG_20230618_0_L2A/catalog.json
Copy node label_S2A_10SGG_20230618_0_L2A from file:///workspaces/s2-labeled/item-label-S2A_10SGG_20230618_0_L2A.json
[Native] Searching for file:///workspaces/s2-labeled/item-label-S2A_10SGG_20230618_0_L2A.json
[Native] resource found at file:///workspaces/s2-labeled/item-label-S2A_10SGG_20230618_0_L2A.json [application/geo+json]
Starting delivery of assets for labels
Delivering asset labels Unknown file:///workspaces/s2-labeled/label-S2A_10SGG_20230618_0_L2A.geojson (S3Streaming) to s3://argo-wfs/svv-dataset/S2A_10SGG_20230618_0_L2A/label-S2A_10SGG_20230618_0_L2A.geojson...
Delivery asset complete to s3://argo-wfs/svv-dataset/S2A_10SGG_20230618_0_L2A/label-S2A_10SGG_20230

2023-08-02 09:20:38.852 | INFO     | utils:ingest_items:236 - Post item label_S2A_10SGG_20230618_0_L2A to https://stac-api-dev.terradue.com//collections/ai-extensions-svv-dataset-labels/items
2023-08-02 09:20:39.661 | INFO     | utils:post_or_put:197 - https://stac-api-dev.terradue.com/collections/ai-extensions-svv-dataset-labels/items 200


https://stac-api-dev.terradue.com/collections/ai-extensions-svv-dataset-labels/items/label_S2A_10SGG_20230618_0_L2A
Stars/2.10.17
Trying to init a root catalog catalog on s3://argo-wfs/svv-dataset/S2A_11SKB_20230618_0_L2A/
Root catalog catalog created at s3://argo-wfs/svv-dataset/S2A_11SKB_20230618_0_L2A/catalog.json
Copy node label_S2A_11SKB_20230618_0_L2A from file:///workspaces/s2-labeled/item-label-S2A_11SKB_20230618_0_L2A.json
[Native] Searching for file:///workspaces/s2-labeled/item-label-S2A_11SKB_20230618_0_L2A.json
[Native] resource found at file:///workspaces/s2-labeled/item-label-S2A_11SKB_20230618_0_L2A.json [application/geo+json]
Starting delivery of assets for labels
Delivering asset labels Unknown file:///workspaces/s2-labeled/label-S2A_11SKB_20230618_0_L2A.geojson (S3Streaming) to s3://argo-wfs/svv-dataset/S2A_11SKB_20230618_0_L2A/label-S2A_11SKB_20230618_0_L2A.geojson...
Delivery asset complete to s3://argo-wfs/svv-dataset/S2A_11SKB_20230618_0_L2A/label-S2A_11SKB_20230

...

https://stac-api-dev.terradue.com/collections/ai-extensions-svv-dataset-labels/items/label_S2A_10SFH_20230618_0_L2A
Stars/2.10.17
Trying to init a root catalog catalog on s3://argo-wfs/svv-dataset/S2B_10SFG_20230613_0_L2A/
Root catalog catalog created at s3://argo-wfs/svv-dataset/S2B_10SFG_20230613_0_L2A/catalog.json
Copy node label_S2B_10SFG_20230613_0_L2A from file:///workspaces/s2-labeled/item-label-S2B_10SFG_20230613_0_L2A.json
[Native] Searching for file:///workspaces/s2-labeled/item-label-S2B_10SFG_20230613_0_L2A.json
[Native] resource found at file:///workspaces/s2-labeled/item-label-S2B_10SFG_20230613_0_L2A.json [application/geo+json]
Starting delivery of assets for labels
Delivering asset labels Unknown file:///workspaces/s2-labeled/label-S2B_10SFG_20230613_0_L2A.geojson (S3Streaming) to s3://argo-wfs/svv-dataset/S2B_10SFG_20230613_0_L2A/label-S2B_10SFG_20230613_0_L2A.geojson...
Delivery asset complete to s3://argo-wfs/svv-dataset/S2B_10SFG_20230613_0_L2A/label-S2B_10SFG_20230

2023-08-02 09:22:02.973 | INFO     | utils:ingest_items:236 - Post item label_S2B_10SFG_20230613_0_L2A to https://stac-api-dev.terradue.com//collections/ai-extensions-svv-dataset-labels/items
2023-08-02 09:22:03.304 | INFO     | utils:post_or_put:197 - https://stac-api-dev.terradue.com/collections/ai-extensions-svv-dataset-labels/items 200


https://stac-api-dev.terradue.com/collections/ai-extensions-svv-dataset-labels/items/label_S2B_10SFG_20230613_0_L2A
Stars/2.10.17
Trying to init a root catalog catalog on s3://argo-wfs/svv-dataset/S2B_10SFH_20230613_0_L2A/
Root catalog catalog created at s3://argo-wfs/svv-dataset/S2B_10SFH_20230613_0_L2A/catalog.json
Copy node label_S2B_10SFH_20230613_0_L2A from file:///workspaces/s2-labeled/item-label-S2B_10SFH_20230613_0_L2A.json
[Native] Searching for file:///workspaces/s2-labeled/item-label-S2B_10SFH_20230613_0_L2A.json
[Native] resource found at file:///workspaces/s2-labeled/item-label-S2B_10SFH_20230613_0_L2A.json [application/geo+json]
Starting delivery of assets for labels
Delivering asset labels Unknown file:///workspaces/s2-labeled/label-S2B_10SFH_20230613_0_L2A.geojson (S3Streaming) to s3://argo-wfs/svv-dataset/S2B_10SFH_20230613_0_L2A/label-S2B_10SFH_20230613_0_L2A.geojson...
Delivery asset complete to s3://argo-wfs/svv-dataset/S2B_10SFH_20230613_0_L2A/label-S2B_10SFH_20230

2023-08-02 09:22:35.389 | INFO     | utils:ingest_items:236 - Post item label_S2B_10SFH_20230613_0_L2A to https://stac-api-dev.terradue.com//collections/ai-extensions-svv-dataset-labels/items
2023-08-02 09:22:35.738 | INFO     | utils:post_or_put:197 - https://stac-api-dev.terradue.com/collections/ai-extensions-svv-dataset-labels/items 200


https://stac-api-dev.terradue.com/collections/ai-extensions-svv-dataset-labels/items/label_S2B_10SFH_20230613_0_L2A
